<a href="https://colab.research.google.com/github/Adityasharma3010/Apache-Beam/blob/course/Composite%20Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install apache_beam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 KB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.5/515.5 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 KB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for crcmod: filename=crcmod-1.7-cp39-cp39-linux_x86_64.whl size=36930 sha256=a4550f5169bbe8949d0f8a2b14413e84c959e8ea7e8fa808e519234122eb1f37
  Stored in directory: /root/.cache/pip/wheels/4a/6c/a6/ff

In [13]:
import apache_beam as beam
p1 = beam.Pipeline()

class MyPTransform(beam.PTransform):
  def expand(self, input_col):
    a = (
        input_col
                  | 'Calculate Sum' >> beam.Map(CalculateSum)
                  | 'Apply Formatting' >> beam.Map(FormatText)
    )
    return a

def SplitRow(input_element):
  return input_element.split(',')

def FilterBasedOnCountry(country_name, input_element):
  return input_element[1] == country_name

def CalculateSum(elem):
  return elem[0], (int(elem[2]) + int(elem[3]) + int(elem[4]))

def FormatText(elem):
  return elem[0] + ' has recieved ' + str(elem[1]) + ' marks.'

input_collection = (
    p1
    | beam.io.ReadFromText('students_marks.txt')
    | beam.Map(SplitRow)
)

US_pipeline = (
    input_collection
    | beam.Filter(lambda record: FilterBasedOnCountry('US', record))
    | 'Calling Ptransform for US' >> MyPTransform()
    | 'Writing results to US File' >> beam.io.WriteToText('US_Result')
)

IN_pipeline = (
    input_collection
    | beam.Filter(lambda record: FilterBasedOnCountry('IN', record))
    | 'Calling Ptransform for India' >> MyPTransform()
    | 'Writing results to India File' >> beam.io.WriteToText('IN_Result')
)

p1.run()

In [14]:
cat US_Result-00000-of-00001

John has recieved 82 marks.
Sam has recieved 89 marks.


In [15]:
cat IN_Result-00000-of-00001

Jim has recieved 100 marks.
Sandhya has recieved 118 marks.
Naina has recieved 126 marks.
